In [2]:
import mfem.ser as mfem
from bfp import *
import numpy as np

In [ ]:
class TotalXSCoefficient(mfem.PyCoefficient):
    """Coefficient for the total cross-section Σ_t(E).

    This coefficient maps the normalized energy coordinate (x[1]) in [0, 1] to the
    corresponding total cross-section value. Specifically, a value of y = 0 corresponds to
    E = E_start and y = 1 corresponds to E = E_end. If a constant is provided 
    (e.g., 1), the coefficient returns this constant at all points.

    Examples:
        TotalXSCoefficient(1)
            # Always returns 1.

        TotalXSCoefficient(xs_t_data, E_start=0.0, E_end=1.0)
            # Returns the interpolated value from xs_t_data.
    """

    def __init__(self, xs_t_data, E_start=None, E_end=None):
        super(TotalXSCoefficient, self).__init__()
        if isinstance(xs_t_data, (int, float)):
            self.constant = True
            self.constant_value = float(xs_t_data)
        else:
            self.constant = False
            self.xs_t_data = xs_t_data
            self.E_start = E_start
            self.E_end = E_end

    def EvalValue(self,ip):
        """Evaluates the total cross-section at a given energy coordinate.

        This method converts the normalized energy coordinate found in x[1] to the actual 
        energy value and determines the corresponding energy group to return the associated 
        total cross-section value. 

        Args:
        ip (list or array-like): A coordinate array where ip[1] is the normalized energy 
                (in the range [0, 1]).

        Returns:
            float: The total cross-section value corresponding to the computed energy.
        """
        if self.constant:
            return self.constant_value

        y = (ip[1] - self.E_start) / (self.E_end - self.E_start)
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.xs_t_data)
        group = min(n_groups - 1,
                    int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.xs_t_data[group])


class ScatteringXSCoefficient(mfem.PyCoefficient):
    """Coefficient for the scattering cross-section Σ_s(E).

    This coefficient maps the normalized energy coordinate (x[1]) in [0, 1] to the
    corresponding cross-section value. Specifically, a value of y = 0 corresponds to
    E = E_start and y = 1 corresponds to E = E_end. If a constant is provided 
    (e.g., 1), the coefficient returns this constant at all points.

    Examples:
        ScatteringXSCoefficient(1)
            # Always returns 1.

        ScatteringXSCoefficient(xs_s_data, E_start=0.0, E_end=1.0)
            # Returns the interpolated value from xs_s_data.
    """

    def __init__(self, xs_s_data, E_start=None, E_end=None):
        super(ScatteringXSCoefficient, self).__init__()
        if isinstance(xs_s_data, (int, float)):
            self.constant = True
            self.constant_value = float(xs_s_data)
        else:
            self.constant = False
            self.xs_s_data = xs_s_data
            self.E_start = E_start
            self.E_end = E_end

    def EvalValue(self, ip):
        """Evaluates the scattering cross-section at a given normalized energy coordinate.

        This method converts the normalized energy coordinate found in x[1] to the actual 
        energy value and determines the corresponding energy group to return the associated 
        scattering cross-section value. 

        Args:
            ip (list or array-like): A coordinate array where ip[1] is the normalized energy 
                in the range [0, 1].

        Returns:
            float: The scattering cross-section value for the computed energy.
        """
        if self.constant:
            return self.constant_value

        y = (ip[1] - self.E_start) / (self.E_end - self.E_start)
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.xs_s_data)
        group = min(n_groups - 1,
                    int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.xs_s_data[group])

class StoppingPowerCoefficient(mfem.PyCoefficient):
    """Coefficient for the stopping power S(E).

    This coefficient maps the normalized energy coordinate (ip[1]) in the interval [0, 1]
    to the corresponding stopping power S(E). Specifically, a value of y = 0 corresponds to 
    E = E_start, and a value of y = 1 corresponds to E = E_end. If a constant is provided 
    (e.g., 1), the coefficient returns this constant at all points.

    Examples:
        StoppingPowerCoefficient(1)
            # Always returns 1.

        StoppingPowerCoefficient(S_data, E_start=0.0, E_end=1.0)
            # Returns the interpolated value from S_data.
    """

    def __init__(self, S_data, E_start=None, E_end=None):
        super(StoppingPowerCoefficient, self).__init__()
        if isinstance(S_data, (int, float)):
            self.constant = True
            self.constant_value = float(S_data)
        else:
            self.constant = False
            self.S_data = S_data
            self.E_start = E_start
            self.E_end = E_end

    def EvalValue(self, ip):
        """Evaluates the stopping power at a given normalized energy coordinate.

        This method converts the normalized energy coordinate found in ip[1] to the actual 
        energy value and determines the corresponding energy group to return the associated 
        stopping power value. 

        Args:
            ip (list or array-like): A coordinate array where ip[1] is the normalized energy 
                (in the range [0, 1]).

        Returns:
            float: The stopping power value corresponding to the computed energy.
        """
        if self.constant:
            return self.constant_value
        y = (ip[1] - self.E_start) / (self.E_end - self.E_start)
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.S_data)
        group = min(n_groups - 1,
                    int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.S_data[group])

class StoppingPowerDerivativeCoefficient(mfem.PyCoefficient):
    """Coefficient for the derivative of the stopping power S(E), i.e., S'(E).

    This coefficient maps the normalized energy coordinate (x[1]) in the interval [0, 1]
    to the corresponding derivative of the stopping power S(E). A value of y = 0 corresponds 
    to E = E_start and y = 1 corresponds to E = E_end. If a constant is provided 
    (e.g., 1), the coefficient returns this constant at all points.

    Examples:
        StoppingPowerDerivativeCoefficient(1)
            # Always returns 1.

        StoppingPowerDerivativeCoefficient(dS_data, E_start=0.0, E_end=1.0)
            # Returns the interpolated value from data_array.
    """

    def __init__(self, dS_data, E_start=None, E_end=None):
        super(StoppingPowerDerivativeCoefficient, self).__init__()
        if isinstance(dS_data, (int, float)):
            self.constant = True
            self.constant_value = float(dS_data)
        else:
            self.constant = False
            self.dS_data = dS_data
            self.E_start = E_start
            self.E_end = E_end

    def EvalValue(self, ip):
        """Evaluates the derivative of the stopping power at a given normalized energy coordinate.

        This method converts the normalized energy coordinate found in ip[1] to the actual 
        energy value and determines the corresponding energy group to return the associated 
        derivative of the stopping power value. 

        Args:
            ip (list or array-like): A coordinate array where ip[1] is the normalized energy 
                (in the range [0, 1]).

        Returns:
            float: The derivative of the stopping power corresponding to the computed energy.
        """
        if self.constant:
            return self.constant_value
        y = (ip[1] - self.E_start) / (self.E_end - self.E_start)
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.dS_data)
        group = min(n_groups - 1,
                    int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.dS_data[group])


class InflowCoefficientSN(mfem.PyCoefficient):
    """Coefficient for an inflow boundary condition in (x,E) space with SN angular dependence.

    This class returns an inflow flux value that depends on the discrete ordinate
    (angular) parameter \(\mu\). On the left boundary:
      - If \(\mu > 0\), the inflow flux is applied (e.g., set to a prescribed value).
      - If \(\mu < 0\), the incoming flux is zero.
    
    This is useful in SN (discrete ordinates) methods where the transport equation is
    solved separately for each angular direction.
    
    Attributes:
        in_flux (float): The constant inflow flux value to be used when \(\mu > 0\).
        mu (float): The discrete ordinate (angular direction) for which this coefficient is used.
    """

    def __init__(self, in_flux, mu):
        """Initializes the SN inflow coefficient.
        
        Args:
            in_flux (float): The constant inflow flux value for \(\mu > 0\).
            mu (float): The discrete angular direction value.
        """
        super(InflowCoefficientSN, self).__init__()
        self.in_flux = in_flux
        self.mu = mu

    def EvalValue(self, ip):
        """Evaluates the inflow coefficient at a given integration point.
        
        In this implementation, the integration point is interpreted in (x,E) space.
        The angular parameter \(\mu\) is stored in the class. The function returns
        the prescribed inflow flux if \(\mu > 0\) and zero otherwise.
        
        Args:
            ip (mfem.IntegrationPoint or list[float]): The integration point coordinates,
                where ip[0] is the spatial coordinate \(x\) and ip[1] is the energy \(E\).
                The angular information is not included in ip.
        
        Returns:
            float: The inflow flux value if \(\mu > 0\); otherwise, 0.0.
        """
        # Since the integration point ip does not include the angular variable,
        # we use the stored self.mu to determine the flux.
        if self.mu > 0:
            return self.in_flux
        else:
            return 0.0


class QCoefficient(mfem.PyCoefficient):
    """Coefficient for the energy dependent source term Q(E). ((will be updated for Q(x,E)))

    This coefficient maps the normalized energy coordinate (ip[1]) in the interval [0, 1]
    to the corresponding the energy dependent source Q(E). Specifically, a value of y = 0 
    corresponds to E = E_start, and a value of y = 1 corresponds to E = E_end. If a constant 
    is provided 
    (e.g., 1), the coefficient returns this constant at all points.

    Examples:
        StoppingPowerCoefficient(1)
            # Always returns 1.

        StoppingPowerCoefficient(S_data, E_start=0.0, E_end=1.0)
            # Returns the interpolated value from S_data.
    """
    def __init__(self, Q_data, E_start=None, E_end=None):
        super(QCoefficient, self).__init__()
        if isinstance(Q_data, (int, float)):
            self.constant = True
            self.constant_value = float(Q_data)
        else:
            self.constant = False
            self.Q_data = Q_data
            self.E_start = E_start
            self.E_end = E_end

    def EvalValue(self, ip):
        """Evaluates the energy dependent source term Q(E) at a given normalized energy coordinate.

        For non-constant Q_data, this method converts the normalized energy coordinate ip[1] to the 
        corresponding energy value and returns the Q value for the associated energy group. 
        If Q_data is constant, it always 
        returns this constant value.

        Args:
            ip (list or array-like): A coordinate array where ip[1] is the normalized energy 
                (in the range [0, 1]).

        Returns:
            float: The energy dependent source value corresponding to the computed energy.
        """

        if self.constant:
            return self.constant_value

        y = (ip[1] - self.E_start) / (self.E_end - self.E_start)
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.Q_data)
        group = min(n_groups - 1, int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.Q_data[group])


class EnergyDependentCoefficient(mfem.PyCoefficient):
    """Energy-dependent coefficient using either a constant or a one-dimensional array.

    This coefficient maps a normalized energy coordinate (ip[1] in [0, 1])
    to a value by linearly interpolating data over an interval defined by E_start and E_end.
    If a constant is provided (e.g., 1), it is converted to an array (with two points)
    and then processed identically to an array input.

    Examples:
        EnergyDependentCoefficient(1)
            # Always returns 1.

        EnergyDependentCoefficient(data_array, E_start=0.0, E_end=1.0)
            # Returns the interpolated value from data_array.
    """

    def __init__(self, data, E_start=None, E_end=None):
        super(EnergyDependentCoefficient, self).__init__()
        if isinstance(data, (int, float)):
            self.constant = True
            self.constant_value = float(data)
        else:
            self.constant = False
            self.data = data
            self.E_start = E_start
            self.E_end = E_end

    def EvalValue(self,ip):
        """Evaluates the coefficient at a given normalized energy coordinate.

        This method converts the normalized energy coordinate (ip[1]) to a physical energy
        value, determines the corresponding index (group) in the data array via linear 
        interpolation, and returns the associated coefficient value.

        Args:
            ip (list or array-like): A coordinate array where ip[1] is the normalized energy 
                (in the range [0, 1]).

        Returns:
            float: The interpolated coefficient value corresponding to the computed energy.
        """
        if self.constant:
            return self.constant_value

        y = (x[1] - self.E_start) / (self.E_end - self.E_start)
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.data)
        group = min(n_groups - 1, int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.data[group])

class XDependentCoefficient(mfem.PyCoefficient):
    """X-dependent coefficient using either a constant or a one-dimensional array.

    This coefficient maps a normalized x coordinate (ip[0] in [0, 1])
    to a value by linearly interpolating data over an interval defined by x_start and x_end.
    If a constant is provided (e.g., 1), it is converted to an array (with two points)
    and then processed identically to an array input.

    Examples:
        XDependentCoefficient(1)
            # Always returns 1.

        XDependentCoefficient(data_array, x_start=0.0, x_end=1.0)
            # Returns the interpolated value from data_array.
    """

    def __init__(self, data, x_start=None, x_end=None):
        super(XDependentCoefficient, self).__init__()
        if isinstance(data, (int, float)):
            self.constant = True
            self.constant_value = float(data)
        else:
            self.constant = False
            self.data = data
            self.x_start = x_start
            self.x_end = x_end

    def EvalValue(self, ip):
        """Evaluates the coefficient at a given normalized x coordinate.

        This method converts the normalized x coordinate (ip[0]) to a physical x
        value, determines the corresponding index (group) in the data array via linear 
        interpolation, and returns the associated coefficient value.

        Args:
            ip (list or array-like): A coordinate array where ip[0] is the normalized x 
                (in the range [0, 1]).

        Returns:
            float: The interpolated coefficient value corresponding to the computed x.
        """
        if self.constant:
            return self.constant_value

        y = (ip[0] - self.x_start) / (self.x_end - self.x_start)
        x_val = self.x_start + y * (self.x_end - self.x_start)
        n_groups = len(self.data)
        group = min(n_groups - 1, int((x_val - self.x_start) / (self.x_end - self.x_start) * n_groups))
        return float(self.data[group])


class VelocityCoefficient(mfem.VectorPyCoefficient):
    """
    A simple vector coefficient:
       v(x) = [mu, S(E)]
    that returns a Python list in EvalValue.
    """
    def __init__(self, mu, S_arr, E_start, E_end):
        super(VelocityCoefficient, self).__init__(2)  # 2D vector
        self.mu = mu
        self.S_arr = S_arr
        self.E_start = E_start
        self.E_end   = E_end

    def EvalValue(self, x):
        """
        x is the coordinate array, e.g. [x_coord, E_coord].
        Returns [mu, S(E)].
        """
        # 1) First component is always mu
        V0 = self.mu
        
        # 2) Convert x[1] to a physical energy E
        E = self.E_start + x[1]*(self.E_end - self.E_start)
        
        # 3) For simplicity, pick an integer index in S_arr
        n_groups = len(self.S_arr)
        idx = min(n_groups - 1,
                  int((E - self.E_start)/(self.E_end - self.E_start) * n_groups))
        
        V1 = self.S_arr[idx]
        
        return [V0, V1]
    
def assign_boundary_attributes(mesh, x_start, E_start, x_end, tol=1e-6):
    """
    Assigns boundary attributes to the mesh:
      - If any vertex of a boundary element is near (x_start, E_start), assign attribute 1.
      - Else if any vertex of a boundary element is near x_end, assign attribute 2.
      - Otherwise, leave the attribute as 0.
    
    Parameters:
      mesh    : The mfem.Mesh object.
      x_start : The x-coordinate for the left (inflow) boundary.
      E_start : The energy coordinate for the left boundary.
      x_end   : The x-coordinate for the right (outflow) boundary.
      tol     : Tolerance for comparing coordinates.
    """
    vertex_array = mesh.GetVertexArray()
    
    # Initialize all boundary element attributes to 0.
    for i in range(mesh.GetNBE()):
        mesh.SetBdrAttribute(i, 0)
    
    # Loop over each boundary element and assign attributes.
    for i in range(mesh.GetNBE()):
        v_indices = mesh.GetBdrElementVertices(i)
        for idx in v_indices:
            x_coord = vertex_array[idx][0]
            E_coord = vertex_array[idx][1]
            if abs(x_coord - x_start) < tol and abs(E_coord - E_start) < tol:
                mesh.SetBdrAttribute(i, 1)
                break
            elif abs(x_coord - x_end) < tol:
                mesh.SetBdrAttribute(i, 2)
                break



class PsiCoefficient(mfem.PyCoefficient):
    """A flexible coefficient that allows setting a user-defined expression.

    This class inherits from mfem.PyCoefficient and evaluates a user-provided
    function (expression) at any given mesh point. The expression is expected
    to be a callable that takes three parameters (x, y, z) and returns a float.
    
    Example usage:
        >>> psi_coeff = PsiCoefficient()
        >>> psi_coeff.set_expression(lambda x, y, z: x**2 + y)
        >>> # Now we can use psi_coeff in GridFunction.ProjectCoefficient, etc.
    """

    def __init__(self, expression=None):
        super().__init__()
        self._expression = expression

    def set_expression(self, expression):
        """Sets or updates the coefficient's underlying expression.

        The provided expression should be a callable that accepts three
        parameters (x, y, z) and returns a float.

        Args:
            expression (Callable[[float, float, float], float]):
                A function that computes the coefficient value at a given point.
        """
        self._expression = expression

    def EvalValue(self, x):
        """Evaluates the coefficient at a given mesh point.

        This method is automatically called by MFEM when computing the
        coefficient's value at each point on the mesh.

        Args:
            x (mfem.Vector): A vector of coordinates representing the point
                at which the coefficient is evaluated. For a 3D problem,
                x[0], x[1], x[2] are the coordinates. In 2D, x[2] may be 0.0.

        Returns:
            float: The value of the expression at the point x.
        """
        if self._expression is None:
            return 0.0
        if len(x) == 1:
            return self._expression(x[0])
        if len(x) == 2:
            return self._expression(x[0], x[1])
        elif len(x) == 3:
            return self._expression(x[0], x[1], x[2])
        else:
            return 0.0


In [39]:
# Set parameters
nx = 2
nE = 3
x_start = 0.0
x_end = 2
E_start = 1
E_end = 0.00
N_ang = 4
order = 0


mesh = create_2D_mesh(nx, nE, x_start, x_end, E_start, E_end)
dim = mesh.Dimension()
#set_boundary_attribute(mesh, -1, x_start, E_start, x_end, tol=1e-8)
"""
vertex_array = mesh.GetVertexArray()
for i in range(mesh.GetNBE()):
    v_indices = mesh.GetBdrElementVertices(i)
    idx0 = v_indices[0]
    vx = vertex_array[idx0][0]
    vE = vertex_array[idx0][1]
    attr = mesh.GetBdrAttribute(i)
    print(f"Boundary element {i}: Vertex = ({vx}, {vE}), Attribute = {attr}")
"""

fec = mfem.DG_FECollection(order, dim)
fes = mfem.FiniteElementSpace(mesh, fec)
Size = fes.GetTrueVSize()
print("Number of unknowns:", Size)

ess_bdr = mfem.intArray(mesh.bdr_attributes.Max())
ess_bdr.Assign(0) 
ess_bdr[0] = 1 # Left boundary,  Dirichlet condition.
ess_bdr[1] = 1 # Right boundary

mu_vals, w_vals = gauss_legendre_dirs(N_ang)

E_arr, E_grid_arr, xs_t_arr, xs_s_arr, S_arr = read_data(50)
S_arr = np.linspace(0, 2, nE + 1)
E_arr = np.linspace(E_start, E_end, nE+1)
xs_t_coeff = TotalXSCoefficient(5)
xs_s_coeff = ScatteringXSCoefficient(5)
S_coeff    = StoppingPowerCoefficient(0)
dS_dE_arr = -compute_S_derivative(E_arr, S_arr) 
dS_dE_coeff = StoppingPowerDerivativeCoefficient(dS_dE_arr, E_start, E_end)
q_coeff = mfem.ConstantCoefficient(100)
inflow_coeff = InflowCoefficient(1.0)
assign_boundary_attributes(mesh, x_start, E_start, x_end, tol=1e-6)

gf = mfem.GridFunction(fes)
gf.Assign(0.0)
psi_coeff = PsiCoefficient()
a, b, c = 1, 2, 3
psi_expr = lambda x, E: a + b*E + c*(x**2)
psi_coeff.set_expression(psi_expr) 
gf.ProjectCoefficient(psi_coeff)



File '/Users/melekderman/github/BFP/examples1/mesh/usr/2x3_2D.mesh' already exists.
Number of unknowns: 6


In [60]:
x_man = np.linspace(x_start, x_end, nx+1)
x_c = (x_man[1:] + x_man[:-1]) / 2
E_man = np.linspace(E_start, E_end, nE+1)
E_c = ((E_man)[1:] + E_man[:-1]) / 2

psi_exact_mu = []

for mu in mu_vals:
    psi_exact = np.zeros((nE, nx))
    for j in range(len(E_c)):
        for i in range(len(x_c)):
            psi_exact = 6 * mu * x_c[i] + (1 + 2 * E_c[j] + 3 * x_c[i] ** 2) * (5 - dS_dE_arr[j]) - S_arr[j] * b
            psi_exact_mu.append((mu, psi_exact))


print(mu_vals)

psi_exact_mu

[-0.8611363115940526, -0.33998104358485626, 0.33998104358485626, 0.8611363115940526]


[(-0.8611363115940526, 7.666591065217842),
 (-0.8611363115940526, 20.499773195653525),
 (-0.8611363115940526, 4.333257731884509),
 (-0.8611363115940526, 17.166439862320193),
 (-0.8611363115940526, 0.9999243985511757),
 (-0.8611363115940526, 13.833106528986859),
 (-0.33998104358485626, 9.23005686924543),
 (-0.33998104358485626, 25.19017060773629),
 (-0.33998104358485626, 5.896723535912098),
 (-0.33998104358485626, 21.85683727440296),
 (-0.33998104358485626, 2.563390202578765),
 (-0.33998104358485626, 18.523503941069624),
 (0.33998104358485626, 11.26994313075457),
 (0.33998104358485626, 31.30982939226371),
 (0.33998104358485626, 7.936609797421236),
 (0.33998104358485626, 27.976496058930376),
 (0.33998104358485626, 4.603276464087902),
 (0.33998104358485626, 24.64316272559704),
 (0.8611363115940526, 12.833408934782158),
 (0.8611363115940526, 36.000226804346475),
 (0.8611363115940526, 9.500075601448824),
 (0.8611363115940526, 32.66689347101314),
 (0.8611363115940526, 6.166742268115492),
 (0

In [ ]:
my_c = StoppingPowerCoefficient(S_data, E_start, E_end)
el_no = 1
el_trans = mesh.GetElementTransformation(el_no)

# 4) Değerlendirmek istediğin integration point'i (örn. Gauss noktalarından biri) seç
ip = mfem.IntegrationPoint()
ip.x = 0
ip.y = -1

# 5) Evaluate
val = my_c.Eval(el_trans, ip)
print(f"Coefficient value at (el={el_no}, ip={ip.x}, {ip.y}) =", val)

Coefficient value at (el=1, ip=0.0, -1.0) = 0.0
